In [1]:
import ipykernel

In [2]:
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import sys
import cupy
print(sys.version)
print(sys.executable)

from sklearn import svm
from skcuda import cudart
from sklearn.preprocessing import StandardScaler
import cupy as cp

!nvidia-smi

ImportError: 
================================================================
Failed to import CuPy.

If you installed CuPy via wheels (cupy-cudaXXX or cupy-rocm-X-X), make sure that the package matches with the version of CUDA or ROCm installed.

On Linux, you may need to set LD_LIBRARY_PATH environment variable depending on how you installed CUDA/ROCm.
On Windows, try setting CUDA_PATH environment variable.

Check the Installation Guide for details:
  https://docs.cupy.dev/en/latest/install.html

Original error:
  ImportError: libcuda.so.1: cannot open shared object file: No such file or directory
================================================================


In [12]:
#1.dataset load
df = pd.read_csv('data.csv')
#pd.set_option('max_columns', None)
#print(list(df.columns))
'''
1 Mild Cognitive Impairment
2 Dementia
3 Subjective Memory Complaint/Normal Cognition
4 Unknown - defer to record
'''
# Filter out unwanted rows and define the label column
df = df[(df['syndrome_v2_v2'] != 4) & (df['syndrome_v2_v2'] != 3)] 
label = 'syndrome_v2_v2' #mild cognitive impairment and Dementia

# Split the data into features and labels
cols_of_interest = ['Left-Caudate',
                    'Right-Caudate',
                    'Left-Putamen',
                    'Left-Pallidum',
                    'Right-Pallidum',
                    'Right-Putamen']
                    #'Animals_raw']
#  'BNT_totalwstim_raw', 'FAS_total_raw', 'TOPF_raw','WTAR_raw'
X = df[cols_of_interest].copy() # Features
y = df[label].copy()


# Encode labels using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

# Combine Xs and y into a single dataframe 
df = pd.concat([X, pd.DataFrame(y, columns = [label])], axis = 1)

# Drop missing values
df.dropna(inplace = True)


# Separate X and y again
X = df[cols_of_interest]
y = df[label]

In [30]:
#2.Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)

#2-1. Move data to GPU
X_train = cp.asarray(X_train)
X_test = cp.asarray(X_test)
y_train = cp.asarray(y_train)
y_test = cp.asarray(y_test)


print(X_train.device)
print(X_test.device)
print(y_train.device)
print(y_test.device)

<CUDA Device 0>
<CUDA Device 0>
<CUDA Device 0>
<CUDA Device 0>


In [26]:
#3.Create an SVM instance, nonlinear model
obj = SVC(C = 1.0, kernel = 'rbf', gamma = 'scale')
model = obj.fit(X_train, y_train)

# obj1 = SVC(C = 1.0, kernel = 'linear', gamma = 'scale')
# model1 = obj1.fit(X_train, y_train)

# obj2 = SVC(C = 1.0, kernel = 'poly', gamma = 'scale')
# model2 = obj1.fit(X_train, y_train)


# obj3 = SVC(C = 1.0, kernel = 'sigmoid', gamma = 'scale')
# model3 = obj1.fit(X_train, y_train)

# obj4 = SVC(C = 1.0, kernel = 'precomputed', gamma = 'scale')
# model4 = obj1.fit(X_train, y_train)

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [ ]:
#4. model evaluation
y_pred = model.predict(X = X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy=', acc)

y_pred = model1.predict(X = X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy=', acc)

y_pred = model2.predict(X = X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy=', acc)

y_pred = model3.predict(X = X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy=', acc)

y_pred = model4.predict(X = X_test)
acc = accuracy_score(y_test, y_pred)
print('accuracy=', acc)

In [ ]:
#5. Grid Search
from sklearn.model_selection import GridSearchCV

params = {'kernel': ['rbf', 'linear', 'poly','sigmoid'],
         'C': [0.01, 0.1, 1.0, 10.0, 100.0],
         'gamma':['scale', 'auto']} 

grid_model = GridSearchCV(model, param_grid = params, 
                         scoring = 'accuracy', cv = 5, n_jobs = -1).fit(X,y)

with cp.cuda.Device(0):
    grid_search.fit(cp.asarray(train_X),cp.asarray(train_y))
                    
                    
                    
# print('best score =', grid_model.best_score_)
# print('best parameters = ', grid_model.best_params_)